In [ ]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm.auto import tqdm
import datetime
import proplot as plot
from wrf import getvar
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

from dask.distributed import Client
client = Client(processes=True, threads_per_worker=1,n_workers=2)
import time
import numpy as np

In [2]:
def do_PCA(array=None):
    skpcaVAR = PCA()
    skpcaVAR.fit(array)
    return skpcaVAR

def forward_diff(arrayin=None,delta=None,axis=None,LT=1):
    result = []
    if axis==0:
            for i in range(0,arrayin.shape[axis]-LT):
                temp = (arrayin[i+LT,:]-arrayin[i,:])/(LT*delta)
                result.append(temp)
            return np.asarray(result)

In [3]:
utestfile = read_and_proc.depickle(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'+'TCGphy/testML/output/haiyan/'+'urad/*')[0])

In [4]:
import importlib
importlib.reload(preproc_haiyan)

path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
#a = preproc_haiyan.preprocess(uvtpath=path+'TCGphy/testML/output/haiyan/',originpath=path+'data/haiyan/',expname=4,window=5,addctrlmethod='new',gaussian=True,sigma=[3,0,0,0],surfix='preproc_dict1b_g').swap_presaved(varname='theta')
[preproc_haiyan.preprocess(uvtpath=path+'TCGphy/testML/output/haiyan/',originpath=path+'data/haiyan/',expname=i,
                          window=5,addctrlmethod='new',gaussian=True,sigma=[3,0,0,0],surfix='preproc_dict1b_g').preproc_uvwF(smooth=True,rmax=utestfile[0].shape[-1]) for i in tqdm(range(1,21))]
[preproc_haiyan.preprocess(uvtpath=path+'TCGphy/testML/output/haiyan/',originpath=path+'data/haiyan/',expname=i,
                               window=5,addctrlmethod='new',gaussian=True,sigma=[3,0,0,0],surfix='preproc_dict1b_g_dudvdwdth').preproc_dudvdw(smooth=True,rmax=utestfile[0].shape[-1]) for i in tqdm(range(1,21))]
[preproc_haiyan.preprocess(uvtpath=path+'TCGphy/testML/output/haiyan/',originpath=path+'data/haiyan/',expname=i,
                           window=5,addctrlmethod='new',gaussian=True,sigma=[3,0,0,0],surfix='preproc_dict1b_g_radcomp').preproc_radcomps(smooth=True,rmax=utestfile[0].shape[-1]) for i in tqdm(range(1,21))]

  0%|          | 0/20 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
np.nan_to_num([[1,1,1],[1,1,np.nan]],nan=10)

array([[ 1.,  1.,  1.],
       [ 1.,  1., 10.]])